<a href="https://colab.research.google.com/github/Nyamu007/SENTIMENT/blob/main/SENTIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

pip install pyforest


  Preparing metadata (setup.py) ... done
  Created wheel for pyforest: filename=pyforest-1.1.2-py2.py3-none-any.whl size=15900 sha256=870666f37e0c771d525f6739a892b523434b9cc0d01b345acbef3d83b7854f39
  Stored in directory: /root/.cache/pip/wheels/c5/ca/73/5cdc3d087111bfbdef90be5457aa03c00c0e32241b2752445c
Successfully built pyforest


In [2]:
import pyforest

In [3]:
file_path = r'C:\Users\User\Desktop\DATA SETS\IMDB Dataset.csv'

In [6]:
from google.colab import files
uploaded = files.upload()




Saving IMDB Dataset.csv to IMDB Dataset.csv


In [7]:
df = pd.read_csv('IMDB Dataset.csv')

In [8]:
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.split()  # Split into words
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]  # Remove stop words
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text]  # Stemming
    return ' '.join(text)

# Apply the cleaning function to the reviews
df['cleaned_review'] = df['review'].apply(clean_text)

# Convert sentiment to numerical
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Split the data
X = df['cleaned_review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [12]:
nltk.download('stopwords')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
pip install nltk

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Train the model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vec)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')


Accuracy: 0.8861
Precision: 0.875
Recall: 0.9029569358999802
F1 Score: 0.8887586678386561


In [15]:
pip install Flask


In [17]:
joblib.dump(model, 'logistic_regression_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
from flask import Flask, request, jsonify
import joblib
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

# Download the stopwords dataset
nltk.download('stopwords')

app = Flask(__name__)

# Load the model and vectorizer
model = joblib.load('logistic_regression_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Text cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.split()
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text]
    return ' '.join(text)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    review = data['review']
    cleaned_review = clean_text(review)
    review_vec = vectorizer.transform([cleaned_review])
    prediction = model.predict(review_vec)
    sentiment = 'positive' if prediction[0] == 1 else 'negative'
    return jsonify({'sentiment': sentiment})

if __name__ == '__main__':
    app.run(debug=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 * Serving Flask app '__init__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
